In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [0]:
IPL_auction_data=spark.read.csv("dbfs:/FileStore/tables/IPL_auction_data.csv", inferSchema=False,header=True)

In [0]:
IPL_auction_data.show()

+---+------------+--------------------+--------------------+----------+-----------+----+
|_c0|     Country|              Player|                Team|Base price|Winning bid|Year|
+---+------------+--------------------+--------------------+----------+-----------+----+
|  0|      Guyana|Christopher Barnwell|Royal Challengers...|      30.5|       30.5|2013|
|  1|South Africa|         Johan Botha|    Delhi Daredevils|       183|      274.5|2013|
|  2|   Australia|    Daniel Christian|Royal Challengers...|        61|         61|2013|
|  3|   Australia|      Michael Clarke| Pune Warriors India|       244|        244|2013|
|  4|   Australia| Nathan Coulter-Nile|      Mumbai Indians|        61|      274.5|2013|
|  5|   Sri Lanka|     Akila Dananjaya| Chennai Super Kings|      12.2|       12.2|2013|
|  6|South Africa|     Quinton de Kock| Sunrisers Hyderabad|      12.2|       12.2|2013|
|  7|    Barbados|       Fidel Edwards|    Rajasthan Royals|        61|      128.1|2013|
|  8|   Australia|   

In [0]:
IPL_auction_data = IPL_auction_data.withColumn("Base price", F.col("Base price").cast("int"))
IPL_auction_data = IPL_auction_data.withColumn("Winning bid", F.col("Winning bid").cast("int"))
IPL_auction_data = IPL_auction_data.withColumn("Year", F.col("Year").cast("int"))

In [0]:
IPL_auction_data.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Player: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- Base price: integer (nullable = true)
 |-- Winning bid: integer (nullable = true)
 |-- Year: integer (nullable = true)



In [0]:
len(IPL_auction_data.columns)

Out[98]: 7

In [0]:
IPL_auction_data.count()

Out[99]: 1052

In [0]:
IPL_auction_data = IPL_auction_data.withColumnRenamed("_c0", "S.No")

In [0]:
IPL_auction_data.show()

+----+------------+--------------------+--------------------+----------+-----------+----+
|S.No|     Country|              Player|                Team|Base price|Winning bid|Year|
+----+------------+--------------------+--------------------+----------+-----------+----+
|   0|      Guyana|Christopher Barnwell|Royal Challengers...|        30|         30|2013|
|   1|South Africa|         Johan Botha|    Delhi Daredevils|       183|        274|2013|
|   2|   Australia|    Daniel Christian|Royal Challengers...|        61|         61|2013|
|   3|   Australia|      Michael Clarke| Pune Warriors India|       244|        244|2013|
|   4|   Australia| Nathan Coulter-Nile|      Mumbai Indians|        61|        274|2013|
|   5|   Sri Lanka|     Akila Dananjaya| Chennai Super Kings|        12|         12|2013|
|   6|South Africa|     Quinton de Kock| Sunrisers Hyderabad|        12|         12|2013|
|   7|    Barbados|       Fidel Edwards|    Rajasthan Royals|        61|        128|2013|
|   8|   A

In [0]:
ipl2023auction=IPL_auction_data.where(F.col('Year') == 2023)

In [0]:
ipl2023auction.show()

+----+------------+----------------+--------------------+----------+-----------+----+
|S.No|     Country|          Player|                Team|Base price|Winning bid|Year|
+----+------------+----------------+--------------------+----------+-----------+----+
| 969| New Zealand| Kane Williamson|      Gujarat Titans|       200|        200|2023|
| 970|     England|     Harry Brook| Sunrisers Hyderabad|       150|       null|2023|
| 971|       India|  Mayank Agarwal| Sunrisers Hyderabad|       100|        825|2023|
| 972|       India|  Ajinkya Rahane| Chennai Super Kings|        50|         50|2023|
| 973|     England|      Sam Curran|        Punjab Kings|       200|       null|2023|
| 974| West Indies|     Odean Smith|      Gujarat Titans|        50|         50|2023|
| 975|    Zimbabwe|   Sikandar Raza|        Punjab Kings|        50|         50|2023|
| 976| West Indies|    Jason Holder|    Rajasthan Royals|       200|        575|2023|
| 977|   Australia|   Cameron Green|      Mumbai India

In [0]:
ipl2023auction.count()

Out[104]: 83

In [0]:
ipl2023auction.count()

Out[105]: 83

In [0]:
avg_winning_bid_by_country = IPL_auction_data.where(F.col('Year') == 2023).groupBy('Country').agg(F.count('Player').alias('no.of_players'))

In [0]:
avg_winning_bid_by_country.show()

+------------+-------------+
|     Country|no.of_players|
+------------+-------------+
| Afghanistan|            1|
|       India|           52|
| West Indies|            5|
|  Bangladesh|            3|
|     Ireland|            1|
|    Zimbabwe|            1|
|     Namibia|            1|
| New Zealand|            2|
|     England|            9|
|   Australia|            4|
|South Africa|            4|
+------------+-------------+



In [0]:
window_spec = Window().partitionBy('Year').orderBy(F.col('Winning bid').desc())
ranked_players = IPL_auction_data.withColumn('BidRank', F.rank().over(window_spec))

In [0]:
ranked_players.show()

+----+-------------------+--------------------+--------------------+----------+-----------+----+-------+
|S.No|            Country|              Player|                Team|Base price|Winning bid|Year|BidRank|
+----+-------------------+--------------------+--------------------+----------+-----------+----+-------+
|  14|          Australia|       Glenn Maxwell|      Mumbai Indians|       122|        610|2013|      1|
|  18|          Sri Lanka|      Ajantha Mendis| Pune Warriors India|        30|        442|2013|      2|
|  29|          Australia|     Kane Richardson| Pune Warriors India|        61|        427|2013|      3|
|  22|              India|      Abhishek Nayar| Pune Warriors India|        61|        411|2013|      4|
|  25|          Sri Lanka|      Thisara Perera| Sunrisers Hyderabad|        30|        411|2013|      4|
|  20|       South Africa|        Chris Morris| Chennai Super Kings|        12|        381|2013|      6|
|  32|          Sri Lanka|Sachithra Senanayake|Kolkata 

In [0]:
ipl2023auction.printSchema()

root
 |-- S.No: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Player: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- Base price: integer (nullable = true)
 |-- Winning bid: integer (nullable = true)
 |-- Year: integer (nullable = true)



In [0]:
window_spec = Window().partitionBy('Year').orderBy(F.col('Winning bid').desc())
ranked_players = ipl2023auction.withColumn('BidRank', F.dense_rank().over(window_spec))

In [0]:
ranked_players.show()

+----+------------+----------------+--------------------+----------+-----------+----+-------+
|S.No|     Country|          Player|                Team|Base price|Winning bid|Year|BidRank|
+----+------------+----------------+--------------------+----------+-----------+----+-------+
| 971|       India|  Mayank Agarwal| Sunrisers Hyderabad|       100|        825|2023|      1|
| 998|       India|     Shivam Mavi|      Gujarat Titans|        40|        600|2023|      2|
| 976| West Indies|    Jason Holder|    Rajasthan Royals|       200|        575|2023|      3|
| 999|       India|    Mukesh Kumar|      Delhi Capitals|        20|        550|2023|      4|
| 980|South Africa|Heinrich Klaasen| Sunrisers Hyderabad|       100|        525|2023|      5|
|1034|South Africa|   Rilee Rossouw|Royal Challengers...|       200|        460|2023|      6|
|1019|     Ireland|     Josh Little|      Gujarat Titans|        50|        440|2023|      7|
|1002|     England|      Will Jacks|Royal Challengers...|   

In [0]:
avg_winning_bid_by_country = ipl2023auction.groupBy('Country').agg(F.round(F.avg('Winning bid'),2).alias('avg_winning_bid'),F.count('Player').alias("no_of_players"))

In [0]:
avg_winning_bid_by_country.show()

+------------+---------------+-------------+
|     Country|avg_winning_bid|no_of_players|
+------------+---------------+-------------+
| Afghanistan|           50.0|            1|
|       India|          83.17|           52|
| West Indies|         193.75|            5|
|  Bangladesh|         116.67|            3|
|     Ireland|          440.0|            1|
|    Zimbabwe|           50.0|            1|
|     Namibia|          100.0|            1|
| New Zealand|          150.0|            2|
|     England|          185.0|            9|
|   Australia|          125.0|            4|
|South Africa|         256.25|            4|
+------------+---------------+-------------+



In [0]:
total_bid_amount = avg_winning_bid_by_country.withColumn('total_winning_bid', F.round(F.col('avg_winning_bid') * F.col('no_of_players'), 2)).orderBy(F.col("total_winning_bid").desc())

In [0]:
result_with_product.show()

+------------+---------------+-------------+-----------------+
|     Country|avg_winning_bid|no_of_players|total_winning_bid|
+------------+---------------+-------------+-----------------+
| Afghanistan|           50.0|            1|             50.0|
|       India|          83.17|           52|          4324.84|
| West Indies|         193.75|            5|           968.75|
|  Bangladesh|         116.67|            3|           350.01|
|     Ireland|          440.0|            1|            440.0|
|    Zimbabwe|           50.0|            1|             50.0|
|     Namibia|          100.0|            1|            100.0|
| New Zealand|          150.0|            2|            300.0|
|     England|          185.0|            9|           1665.0|
|   Australia|          125.0|            4|            500.0|
|South Africa|         256.25|            4|           1025.0|
+------------+---------------+-------------+-----------------+



In [0]:
percentage_increase = ipl2023auction.withColumn('PercentageIncrease', ((F.col('Winning bid') - F.col('Base price')) / F.col('Base price')) * 100).orderBy(F.col("PercentageIncrease").desc()).show()

+----+------------+--------------------+--------------------+----------+-----------+----+------------------+
|S.No|     Country|              Player|                Team|Base price|Winning bid|Year|PercentageIncrease|
+----+------------+--------------------+--------------------+----------+-----------+----+------------------+
| 999|       India|        Mukesh Kumar|      Delhi Capitals|        20|        550|2023|            2650.0|
| 998|       India|         Shivam Mavi|      Gujarat Titans|        40|        600|2023|            1400.0|
| 989|       India|      Vivrant Sharma| Sunrisers Hyderabad|        20|        260|2023|            1200.0|
|1010|       India|        Mayank Dagar| Sunrisers Hyderabad|        20|        180|2023|             800.0|
|1012|       India|       Prerak Mankad|Lucknow Super Giants|        20|        180|2023|             800.0|
|1019|     Ireland|         Josh Little|      Gujarat Titans|        50|        440|2023|             780.0|
| 971|       India|

In [0]:
max_winning_bid_players = IPL_auction_data.withColumn('BidRank', F.rank().over(window_spec)).filter('BidRank == 1').show()

+----+------------+------------------+--------------------+----------+-----------+----+-------+
|S.No|     Country|            Player|                Team|Base price|Winning bid|Year|BidRank|
+----+------------+------------------+--------------------+----------+-----------+----+-------+
|  14|   Australia|     Glenn Maxwell|      Mumbai Indians|       122|        610|2013|      1|
|  39|     England|   Kevin Pietersen|    Delhi Daredevils|       200|        900|2014|      1|
| 199|       India|      Yuvraj Singh|    Delhi Daredevils|       200|       1600|2015|      1|
| 260|   Australia|      Shane Watson|Royal Challengers...|       200|        950|2016|      1|
| 354|     England|        Ben Stokes|Rising Pune Super...|       200|       1450|2017|      1|
| 420|     England|        Ben Stokes|    Rajasthan Royals|       200|       1250|2018|      1|
| 595|       India|    Jaydev Unadkat|    Rajasthan Royals|       150|        840|2019|      1|
| 605|       India|Varun Chakravarthy|  

In [0]:
window_spec_cumulative = Window().partitionBy('Player').orderBy('Year').rowsBetween(Window.unboundedPreceding, 0)
cumulative_winning_bid = IPL_auction_data.withColumn('CumulativeWinningBid', F.sum('Winning bid').over(window_spec_cumulative))


In [0]:
cumulative_winning_bid.show(300)

+----+--------------------+--------------------+--------------------+----------+-----------+----+--------------------+
|S.No|             Country|              Player|                Team|Base price|Winning bid|Year|CumulativeWinningBid|
+----+--------------------+--------------------+--------------------+----------+-----------+----+--------------------+
|  53|           Australia|         Aaron Finch| Sunrisers Hyderabad|       100|        400|2014|                 400|
| 222|           Australia|         Aaron Finch|      Mumbai Indians|       100|        320|2015|                 720|
| 307|           Australia|         Aaron Finch|       Gujarat Lions|       100|        100|2016|                 820|
| 434|           Australia|         Aaron Finch|     Kings XI Punjab|       150|        620|2018|                1440|
| 650|           Australia|         Aaron Finch|Royal Challengers...|       100|        440|2020|                1880|
|1046|               India|        Abdul Basith|

In [0]:
window_spec_cumulative = Window().partitionBy('Team').orderBy('Year').rowsBetween(Window.unboundedPreceding, 0)
cumulative_winning_bid_by_team = IPL_auction_data.withColumn('CumulativeWinningBid', F.sum('Winning bid').over(window_spec_cumulative))


In [0]:
cumulative_winning_bid_by_team.show(300)

+----+--------------------+--------------------+-------------------+----------+-----------+----+--------------------+
|S.No|             Country|              Player|               Team|Base price|Winning bid|Year|CumulativeWinningBid|
+----+--------------------+--------------------+-------------------+----------+-----------+----+--------------------+
|   5|           Sri Lanka|     Akila Dananjaya|Chennai Super Kings|        12|         12|2013|                  12|
|  11|            Barbados|        Jason Holder|Chennai Super Kings|        12|         12|2013|                  24|
|  13|           Australia|        Ben Laughlin|Chennai Super Kings|        12|         12|2013|                  36|
|  20|        South Africa|        Chris Morris|Chennai Super Kings|        12|        381|2013|                 417|
|  21|           Australia|         Dirk Nannes|Chennai Super Kings|       122|        366|2013|                 783|
|  45|        South Africa|      Faf du Plessis|Chennai 

In [0]:
amount_spent_by_team=ipl2023auction.groupBy('Team').agg(F.sum("Winning bid").alias("total_amount_spent"))

In [0]:
amount_spent_by_team.show()

+--------------------+------------------+
|                Team|total_amount_spent|
+--------------------+------------------+
| Sunrisers Hyderabad|              2245|
|Lucknow Super Giants|               540|
| Chennai Super Kings|               270|
|      Gujarat Titans|              1480|
|    Rajasthan Royals|              1075|
|Royal Challengers...|              1110|
|Kolkata Knight Ri...|               690|
|        Punjab Kings|               150|
|      Delhi Capitals|              1040|
|      Mumbai Indians|               300|
+--------------------+------------------+



In [0]:
no_players_by_team=ipl2023auction.groupBy('Team').agg(F.count("Player").alias("No_of_players"))

In [0]:
no_players_by_team.show()

+--------------------+-------------+
|                Team|No_of_players|
+--------------------+-------------+
| Sunrisers Hyderabad|           13|
|Lucknow Super Giants|           10|
| Chennai Super Kings|            7|
|      Gujarat Titans|            7|
|    Rajasthan Royals|           11|
|Royal Challengers...|            8|
|Kolkata Knight Ri...|            9|
|        Punjab Kings|            6|
|      Delhi Capitals|            4|
|      Mumbai Indians|            8|
+--------------------+-------------+

